<a href="https://colab.research.google.com/github/mavito/NLP/blob/master/M2_NLP_J053.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [0]:
import io
import numpy as np
import regex as re
import nltk
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import gensim
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from sklearn.decomposition import NMF
import spacy
from nltk.stem import WordNetLemmatizer

In [19]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
stopword = set(stopwords.words('german'))

In [0]:
spac = spacy.load('de')

In [61]:
!wget https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true

--2020-03-03 18:28:42--  https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:28:42--  https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:28:42--  https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.12

In [62]:
!mkdir newsarticles
!unzip 10k-german-news-articles.zip?raw=true -d newsarticles

Archive:  10k-german-news-articles.zip?raw=true
  inflating: newsarticles/Annotations.csv  
  inflating: newsarticles/Annotations_consolidated.csv  
  inflating: newsarticles/Articles.csv  
  inflating: newsarticles/Categories.csv  
  inflating: newsarticles/CrossValSplit.csv  
  inflating: newsarticles/Newspaper_Staff.csv  
  inflating: newsarticles/Posts.csv  


In [0]:
zipfile = ZipFile('/content/10k-german-news-articles.zip?raw=true','r')

In [67]:
zipfile.namelist()

['Annotations.csv',
 'Annotations_consolidated.csv',
 'Articles.csv',
 'Categories.csv',
 'CrossValSplit.csv',
 'Newspaper_Staff.csv',
 'Posts.csv']

In [0]:
anno = pd.read_csv(zipfile.open('Annotations.csv'),header = 'infer')
annoconso = pd.read_csv(zipfile.open('Annotations_consolidated.csv'),header = 'infer')
articles = pd.read_csv(zipfile.open('Articles.csv'),header = 'infer')
categ = pd.read_csv(zipfile.open('Categories.csv'),header = 'infer')
crossval = pd.read_csv(zipfile.open('CrossValSplit.csv'),header = 'infer')
news = pd.read_csv(zipfile.open('Newspaper_Staff.csv'),header = 'infer')
posts = pd.read_csv(zipfile.open('Posts.csv'),header = 'infer')

In [0]:
listallcsv = [anno,annoconso,articles,categ,crossval,news,posts]

In [35]:
#HERE WE FIND OUT ALL THE NULL VALUES PER CSV FILE AND SOME BASIC INFO
print('\nAnno\n',anno.isna().sum(),anno.columns,anno.shape)
print('\nAnnoConso\n',annoconso.isna().sum(),annoconso.columns,annoconso.shape)
print('\nArticles\n',articles.isna().sum(),articles.columns,articles.shape)
print('\nCategories\n',categ.isna().sum(),categ.columns,categ.shape)
print('\nCrossVal\n',crossval.isna().sum(),crossval.columns,crossval.shape)
print('\nNews\n',news.isna().sum(),news.columns,news.shape)
print('\nPosts\n',posts.isna().sum(),posts.columns,posts.shape)


Anno
 ID_Post         0
ID_Annotator    0
Category        0
Value           0
dtype: int64 Index(['ID_Post', 'ID_Annotator', 'Category', 'Value'], dtype='object') (58568, 4)

AnnoConso
 ID_Post     0
Category    0
Value       0
dtype: int64 Index(['ID_Post', 'Category', 'Value'], dtype='object') (40567, 3)

Articles
 ID_Article        0
Path              0
publishingDate    0
Title             0
Body              0
dtype: int64 Index(['ID_Article', 'Path', 'publishingDate', 'Title', 'Body'], dtype='object') (12087, 5)

Categories
 Name    0
Ord     0
dtype: int64 Index(['Name', 'Ord'], dtype='object') (9, 2)

CrossVal
 ID_Post     0
Category    0
Fold        0
dtype: int64 Index(['ID_Post', 'Category', 'Fold'], dtype='object') (40567, 3)

News
 ID_User    0
dtype: int64 Index(['ID_User'], dtype='object') (110, 1)

Posts
 ID_Post                0
ID_Parent_Post    317007
ID_Article             0
ID_User                0
CreatedAt              0
Status                 0
Headline        

In [0]:
#Merge posts.csv and Annotation_Consolidated.csv
result_df = pd.merge(posts, annoconso, on="ID_Post")

In [0]:
#merge the previous df with Articles.csv
dfm = pd.merge(result_df, articles, on="ID_Article")

In [38]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40567 entries, 0 to 40566
Data columns (total 16 columns):
ID_Post           40567 non-null int64
ID_Parent_Post    26554 non-null float64
ID_Article        40567 non-null int64
ID_User           40567 non-null int64
CreatedAt         40567 non-null object
Status            40567 non-null object
Headline          13650 non-null object
Body_x            38839 non-null object
PositiveVotes     40567 non-null int64
NegativeVotes     40567 non-null int64
Category          40567 non-null object
Value             40567 non-null int64
Path              40567 non-null object
publishingDate    40567 non-null object
Title             40567 non-null object
Body_y            40567 non-null object
dtypes: float64(1), int64(6), object(9)
memory usage: 5.3+ MB


In [39]:
#As given in Category.csv, every category has a specific order of priority, so replace category name with order
for j in range(dfm.shape[0]):
  for i in range(categ.shape[0]):
    if dfm['Category'][j]==categ['Name'][i]:
      dfm['Category'][j]=categ['Ord'][i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [40]:
dfm #IS OUR DENORMALISED dataset with all necessary columns
#CSVs used until now: Posts, Annotation, Articles ,Category

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body_x,PositiveVotes,NegativeVotes,Category,Value,Path,publishingDate,Title,Body_y
0,79,NaN,1,12071,2015-06-01 08:58:32.363,online,NaN,"ich kann keinen hinweis finden, wo man sich hi...",0,0,7,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,81,NaN,1,27465,2015-10-03 14:06:06.210,online,NaN,Sehr geehrte Community oder cmb!\r\n\r\nBekomm...,0,1,7,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
2,132,NaN,2,28073,2015-10-09 11:33:07.227,online,Irgendwie...,kommen manche Posts nicht durch :(,0,0,7,1,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
3,134,NaN,2,20194,2015-11-04 13:29:34.930,online,NaN,Eine Frage: \r\n\r\nGibt es eine Möglichkeit s...,0,0,7,1,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
4,139,NaN,2,14020,2015-12-11 14:51:17.540,online,Registrierung,"Ich habe ein Problem: ich bin registriert, übe...",0,0,7,1,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40562,1006960,1006958.0,12070,24704,2016-05-31 16:37:21.110,online,Früher,haben sie ein vierteltelefon beantragen dürfen...,5,3,1,1,Newsroom/Web/Telekommunikation/mobilc,2016-05-31 14:37:48.00,Neue A1-Chefin Schramböck rührt im Management um,"<div class=""section"" id=""content-main"" itempro..."
40563,1006960,1006958.0,12070,24704,2016-05-31 16:37:21.110,online,Früher,haben sie ein vierteltelefon beantragen dürfen...,5,3,2,0,Newsroom/Web/Telekommunikation/mobilc,2016-05-31 14:37:48.00,Neue A1-Chefin Schramböck rührt im Management um,"<div class=""section"" id=""content-main"" itempro..."
40564,1006960,1006958.0,12070,24704,2016-05-31 16:37:21.110,online,Früher,haben sie ein vierteltelefon beantragen dürfen...,5,3,3,0,Newsroom/Web/Telekommunikation/mobilc,2016-05-31 14:37:48.00,Neue A1-Chefin Schramböck rührt im Management um,"<div class=""section"" id=""content-main"" itempro..."
40565,1010868,1010862.0,12079,14509,2016-05-31 18:24:10.780,online,NaN,"In dem Artikel, unter dem du dein post abgeset...",6,1,7,0,Newsroom/Web/Innovationen/Microsoft,2016-05-31 09:43:18.00,"Windows-10-Upgrade: Microsoft will ""üble Trick...","<div class=""section"" id=""content-main"" itempro..."


In [41]:
dfm.info()#We Now Extract Topics from Title and Body and compare

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40567 entries, 0 to 40566
Data columns (total 16 columns):
ID_Post           40567 non-null int64
ID_Parent_Post    26554 non-null float64
ID_Article        40567 non-null int64
ID_User           40567 non-null int64
CreatedAt         40567 non-null object
Status            40567 non-null object
Headline          13650 non-null object
Body_x            38839 non-null object
PositiveVotes     40567 non-null int64
NegativeVotes     40567 non-null int64
Category          40567 non-null object
Value             40567 non-null int64
Path              40567 non-null object
publishingDate    40567 non-null object
Title             40567 non-null object
Body_y            40567 non-null object
dtypes: float64(1), int64(6), object(9)
memory usage: 6.5+ MB


In [0]:
#remove html tags
def remove_html(x):
  return BeautifulSoup(x, "lxml").text

#remove \n from sentences
def remove_slash_n(x):
  return re.sub('\n', " ",x)

#basic cleaning
def clean_text(x):
  x = x.rstrip()
  sentence = x.lower()
  S=re.sub('"','',sentence)
  S = re.sub(r'[^\w]', ' ', S)
  S=re.sub('[^a-zA-Z]', " ", S)
  stop_free = ' '.join([word for word in S.lower().split() if ((word not in stopword))])
  return stop_free

#german lemmatizer spacy
def lemmatazer(x):
  sentence = ' '.join([t.lemma_ for t in spac.tokenizer(x)])
  return sentence

#culmination of all
def preprocessing_text(x):
  x=remove_html(x)
  x=remove_slash_n(x)
  x=clean_text(x)
  x=lemmatazer(x)
  return x

In [0]:
#applying text preprocessing to required columns
dfm['CleanBody'] = dfm['Body_y'].apply(preprocessing_text)
dfm['CleanTitle'] = dfm['Title'].apply(preprocessing_text)

#TOPIC MODELLING DONE BY NMF INSTEAD OF LDA, PROVIDES BETTER RESULTS

FOR TITLE

In [0]:
message=[]
message2=[]

In [0]:
vectorizer1 = TfidfVectorizer()
X = vectorizer1.fit_transform(dfm['CleanTitle'])

In [0]:
idx_to_word1 = np.array(vectorizer1.get_feature_names())

In [0]:
nmf1 = NMF(n_components=20, solver="mu")
W1 = nmf1.fit_transform(X)
H1 = nmf1.components_

In [0]:
for i, topic in enumerate(H1):
  message.append([str(x) for x in idx_to_word1[topic.argsort()[-10:]]])

FOR BODY

In [0]:
vectorizer2 = TfidfVectorizer()

In [0]:
Y = vectorizer2.fit_transform(dfm['CleanBody'])

In [0]:
idx_to_word2 = np.array(vectorizer2.get_feature_names())

In [0]:
nmf2 = NMF(n_components=20, solver="mu")
W2 = nmf2.fit_transform(Y)
H2 = nmf2.components_

In [0]:
for i, topic in enumerate(H2):
    message2.append([str(x) for x in idx_to_word2[topic.argsort()[-10:]]])

#SIMILARITY BETWEEN TITLE AND BODY

FOR BODY and title

In [0]:
message=pd.DataFrame(message).add_prefix('word_')
message2=pd.DataFrame(message2).add_prefix('word_')

In [55]:
message

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,modeimperiums,wunsch,lehrstellen,fu,chtling,balltrainer,fall,realit,tsverlust,genderwahn
1,babys,schlafen,gen,slowakische,kleinstadt,kopftuchkonflikte,traiskirchen,iran,kinderarbeiter,zeltbewohner
2,verbilligen,einsperren,fl,chtlinge,erneut,polizei,setzen,idomeni,tr,nengas
3,stehen,reisen,entl,pal,sst,israel,frei,ngste,stinensische,gefangen
4,griss,upl,ball,blatter,alternativ,fu,warum,bleiben,gewalt,partnerschaft
5,eisenstange,frau,pal,israelisch,verletzen,tet,get,soldat,stinenser,kopfschuss
6,chtigen,tatverd,erst,gest,ndnis,praterstern,vergewaltigung,ftling,vorfall,verlegen
7,neuformulierung,euro,millionen,bekommen,new,york,schwedin,sexuell,stigung,bel
8,mariahilfer,bau,abschließen,zufrieden,israelisch,neu,gesetz,recht,siedlungen,ausdehnen
9,deutschland,sterreich,tickets,einstellen,verbilligen,fl,chtlinge,junge,angeblich,egoisten


In [57]:
message2

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,hausbichler,matriarchats,zeigen,laut,scheinen,ngst,ltnisse,feminismus,tun,kritik
1,acht,richtig,junge,asylwerber,hriger,afghanistan,iran,sami,chte,zelt
2,mazedonisch,hilfen,rkei,versuchen,grenze,sein,mitrow,eu,chtlinge,fl
3,zumeist,jemals,wawi,strafm,milit,hrige,stinensische,lfj,zw,pal
4,fragen,beziehung,frauenh,warum,frauen,partner,tigen,betroffen,gewaltt,gewalt
5,ver,kopf,ffentlicht,armee,menschenrechtsorganisation,video,verletzen,israelisch,betselem,soldat
6,apa,jugendlich,attacke,freitag,nieder,martin,verlegen,justizministerium,praterstern,tichy
7,new,gericht,gesp,york,digung,entsch,finanzen,schwedin,chef,millionen
8,spielen,gewinnen,spa,deutschen,ikbal,fu,grenzen,kicken,gst,ttner
9,arbeit,jahr,verm,netto,poor,working,besteuern,wer,trotzen,mittelschicht


In [56]:
j = 0
for i in range(10):
  topic_title = " ".join(message.values[i])
  topic_body = " ".join(message2.values[i])

  ttt = spac(topic_title)
  ttb = spac(topic_body)
  print(ttt.similarity(ttb))

/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if

0.8492715003521002
0.7073257208685212
0.880438770822803
0.7337482763674271
0.7817328936946805
0.8328940661922916
0.8183525717800229
0.8982939901568343
0.6830815138251166
0.6812740005250405


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if

In [0]:
messagel = [' '.join(message.values[i]) for i in range(len(message))]

In [0]:
messagel2 = [' '.join(message2.values[i]) for i in range(len(message2))]

In [60]:
for i in range(len(message)):
  ttt = spac(messagel[i])
  ttb = spac(messagel2[i])
  print('Similarity % : ',ttt.similarity(ttb))

/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if

Similarity % :  0.8492715003521002
Similarity % :  0.7073257208685212
Similarity % :  0.880438770822803
Similarity % :  0.7337482763674271
Similarity % :  0.7817328936946805
Similarity % :  0.8328940661922916
Similarity % :  0.8183525717800229


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if

Similarity % :  0.8982939901568343
Similarity % :  0.6830815138251166
Similarity % :  0.6812740005250405
Similarity % :  0.7427722219601893
Similarity % :  0.8426403050652811
Similarity % :  0.8766047184837708
Similarity % :  0.7816973647778667
Similarity % :  0.7429175426283856
Similarity % :  0.8102325536286152
Similarity % :  0.7524006083487659
Similarity % :  0.6037463718701144
Similarity % :  0.7970408736642379
Similarity % :  0.9033150572148319


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if

#Sheet Done, M2 Done , F